## Table of Contents

- [xmsmesh "hello world"](#xmsmesh-"hello-world")
- [Define the Polygon Points and Display](#Define-the-Polygon-Points-and-Display)
- [Create the Mesh Inputs and Generate the Mesh](#Create-the-Mesh-Inputs-and-Generate-the-Mesh)
- [Display the Mesh](#Display-the-Mesh)
- [Read polygon from shapefile](#Read-polygon-from-shapefile)
- [xms.mesher.api generate_mesh()](#xms.mesher.api-generate_mesh())
- [RefinePoint](#RefinePoint)

## xmsmesh "hello world"
This is an introduction to the xmsmesh library. This library generates unstructured triangular or mixed triangle, quad meshes. The meshes are generated from input polygons. The resulting mesh will honor the input polygon boundaries. The density of mesh elements is controlled by the spacing of points along the polygon boundaries.

In addition, the density of elements can be controlled by specifying a size function (this is demonstrated in another notebook {see the examples folder}).

This example shows how to generate a mesh from a simple, square polygon with coordinates (0,0), (100,100) and a boundary spacing of 10.0.

In [1]:
import numpy as np
import meshing_tools
from xms import mesher
from xms.grid.ugrid import ugrid_utils

## Define the Polygon Points and Display

In [2]:
# Define the polygon points
polygon_points = np.array([
    (0, 10,0),   (0, 20,0),   (0, 30,0),   (0, 40,0),    (0, 50,0),   (0, 60,0),   (0, 70,0),   (0, 80,0),
    (0, 90,0),   (0, 100,0),  (10, 100,0), (20, 100,0),  (30, 100,0), (40, 100,0), (50, 100,0), (60, 100,0),
    (70, 100,0), (80, 100,0), (90, 100,0), (100, 100,0), (100, 90,0), (100, 80,0), (100, 70,0), (100, 60,0),
    (100, 50,0), (100, 40,0), (100, 30,0), (100, 20,0),  (100, 10,0), (100, 0,0),  (90, 0,0),   (80, 0,0),
    (70, 0,0),   (60, 0,0),   (50, 0,0),   (40, 0,0),    (30, 0,0),   (20, 0,0),   (10, 0,0),   (0, 0,0)
])

The code below uses holoviews to create an interactive render of our polygon points. The resulting render is shown in the image that follows.

```python
# Display the Polygon
# We are using holoviews instead of geoviews here because we don't need projections for this.
%opts Polygons Points [width=500, height=500]
polygon = Polygon(polygon_points)
polygons = hv.Polygons([polygon,]).redim.range(x=(-15, 115), y=(-15, 115))
points = hv.Points(polygon_points).options(color='black', size=6)
polygons * points
```

![Polygon created from above points](images/polygon_points.png)

## Create the Mesh Inputs and Generate the Mesh

In [3]:
# Create Meshing Inputs
input_polygon = mesher.meshing.PolyInput(outside_polygon=polygon_points)
mesh_io = mesher.meshing.MultiPolyMesherIo(polygons=[input_polygon])

# Generate Mesh
succeded, errors = mesher.meshing.mesh_utils.generate_mesh(mesh_io=mesh_io)
if succeded:
    print("Meshing was successful")
else:
    print("Meshing errors found:")
    print("\t{}".format(errors))


Meshing was successful


## Display the Mesh

The code below uses holoviews to generate a render of our mesh. The resulting render is shown in the following image.

```python
pts, cells = meshing_tools.xmsmesh_to_dataframe(mesh_io.points, mesh_io.cells)
vert_points = hv.Points(pts, vdims=['z'])
trimesh = hv.TriMesh((cells, vert_points))
polygons * points * trimesh.edgepaths.options(line_width=0.5)
```

![The mesh that was generated from the above code](images/mesh.png)

## Read polygon from shapefile

In [4]:
from shapely.geometry import shape
import fiona
shape_file = fiona.open("square_holes_polys.shp")
first = next(iter(shape_file))
shp_geom = shape(first['geometry'])

x, y = shp_geom.exterior.coords.xy
coords = np.array(list(zip(x, y)))

holes = []
for hole in shp_geom.interiors:
    xh, yh = hole.coords.xy
    hole_coords = np.array(list(zip(xh, yh)))
    holes.append(hole_coords)

# Create Meshing Inputs
input_polygon = mesher.meshing.PolyInput(outside_polygon=coords[:-1], inside_polygons=[x[:-1] for x in holes])
mesh_io = mesher.meshing.MultiPolyMesherIo(polygons=[input_polygon])

# Generate Mesh
succeded, errors = mesher.meshing.mesh_utils.generate_mesh(mesh_io=mesh_io)
if succeded:
    print("Meshing was successful")
else:
    print("Meshing errors found:")
    print("{}".format(errors))

Meshing was successful


The code below generates a render of our mesh using holowviews. The resulting render is shown in the image below.

```python
polygon_w_hole = Polygon(coords, holes)
polygons_holes = hv.Polygons([polygon_w_hole,]).redim.range(x=(-15, 115), y=(-15, 115))

pts, cells = meshing_tools.xmsmesh_to_dataframe(mesh_io.points, mesh_io.cells)
vert_points = hv.Points(pts, vdims=['z'])
trimesh = hv.TriMesh((cells, vert_points))
polygons_holes * trimesh.edgepaths.options(line_width=0.5)
```

![The mesh that was generated from the above code](images/mesh_2.png)

## xms.mesher.api generate_mesh()

xmsmesher provides a function called `generate_mesh()` that allows you to generate a mesh with a single function call. It takes two arguments which are shown below:

- `polygon_inputs` (list of PolyInput): The polygon inputs for meshing.
- `refine_points` (list of RefinePoint): A list of RefinePoint's used in meshing. (Optional)

`refine_points` is optional, and we will go over `RefinePoint` later. `generate_mesh()` returns the mesh as an xmsgrid `UGrid` object. An example on how to use `generate_mesh()` is given below.

In [5]:
# Import needed for generate_mesh()
from xms.mesher.api.mesh import generate_mesh
from xms.grid.ugrid import ugrid_utils

# Generate mesh from data created earlier in the notebook
uGrid = generate_mesh(polygon_inputs=[input_polygon])

# Write the UGrid out to a file
ugrid_utils.write_ugrid_to_ascii_file(uGrid, 'output/mesh_api.xmugrid')

uGrid

<UGrid - Number of Locations: 116, Number of Cells: 140, Extents: ((0.0, 0.0, 0.0), (100.0, 100.0, 0.0)), Modified: True>

The image below is a screenshot of our UGrid when imported into GMS.

![](images/mesh_api.png)

## RefinePoint

xmsmesher provides the `RefinePoint` class to allow you to create refine points for your mesh. A refine point is a feature point that is created inside the boundary of a polygon and assigned a size value. When the finite element mesh is created, a corner node will be created at the location of the refine point and all element edges that touch the node will be the exact length specified by the refine point size value. The `RefinePoint` constructor takes the following three arguments:

- `point` (Tuple): The location of the refine point. An (x,y,z) tuple. 
- `size` (Real): The element edge length in the vicinity of the refine point.
- `create_mesh_point` (Bool): Whether or not to create a corner node at the location of this refine point. (Optional, defaults to `True`)

We can use `RefinePoint` when generating a mesh with `MultiPolyMesherIo` as shown below:

In [6]:
# Import needed so we can write our mesh out to a file as a UGrid
from xms import grid

# Create Meshing Inputs
input_polygon = mesher.meshing.PolyInput(outside_polygon=polygon_points)
refine_point = mesher.meshing.RefinePoint(point=(13.0, 50.0, 0.0), size=0.5, create_mesh_point=True)
mesh_io = mesher.meshing.MultiPolyMesherIo(polygons=[input_polygon], refine_points=[refine_point])

# Generate Mesh
succeded, errors = mesher.meshing.mesh_utils.generate_mesh(mesh_io=mesh_io)
if succeded:
    print("Meshing was successful")
    grid = grid.ugrid.UGrid(points=mesh_io.points, cellstream=mesh_io.cells)
    ugrid_utils.write_ugrid_to_ascii_file(grid, "output/ugrid_1.xmugrid")
else:
    print("Meshing errors found:")
    print("\t{}".format(errors))

Meshing was successful


Our generated mesh with our refine point is shown below.

![Our mesh with a refine point](images/mesh_refine_1.png)

We can also use `RefinePoint` with the xms.mesher.api `generate_mesh()` function as shown below.

In [7]:
# Create Meshing Inputs
input_polygon = mesher.meshing.PolyInput(outside_polygon=polygon_points)
refine_point = mesher.meshing.RefinePoint(point=(50.0, 50.0, 0.0), size=0.5, create_mesh_point=True)

# Generate the mesh
ugrid = generate_mesh(polygon_inputs=[input_polygon], refine_points=[refine_point])

# Write the mesh out to a file
ugrid_utils.write_ugrid_to_ascii_file(ugrid, "output/ugrid_2.xmugrid")

![](images/mesh_refine_2.png)